In [1]:

import pandas as pd
import numpy as np
from mealpy.optimizer import Optimizer

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

import numpy as np
import time  # Import the time module

from mealpy.utils import io
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Assuming you have a dataset 'X' and corresponding labels 'y'

# Assuming you have a dataset 'X' and corresponding labels 'y'

datasetOZ = pd.read_csv('labelled-balanced-data11.csv')
X, y = datasetOZ.iloc[:, :-1], datasetOZ.iloc[:, -1]

# Determine the number of features
num_features = X.shape[1]

In [2]:
class OriginalZOA(Optimizer):
    def __init__(self, epoch=10000, pop_size=100, **kwargs):
        super().__init__(**kwargs)
        self.epoch = self.validator.check_int("epoch", epoch, [1, 100000])
        self.pop_size = self.validator.check_int("pop_size", pop_size, [10, 10000])
        self.set_parameters(["epoch", "pop_size"])
        self.sort_flag = False
        
    def evolve(self, epoch):
        
        # PHASE1: Foraging Behaviour
        pop_new = []
        for idx in range(0, self.pop_size):
            r1 = np.round(1 + np.random.rand())
            pos_new = self.pop[idx][self.ID_POS] + np.random.rand(self.problem.n_dims) * (self.g_best[self.ID_POS] - r1 * self.pop[idx][self.ID_POS])   # Eq. 3
            pos_new = self.amend_position(pos_new, self.problem.lb, self.problem.ub)
            pop_new.append([pos_new, None])
            if self.mode not in self.AVAILABLE_MODES:
                target = self.get_target_wrapper(pos_new)
                self.pop[idx] = self.get_better_solution([pos_new, target], self.pop[idx])
        if self.mode in self.AVAILABLE_MODES:
            pop_new = self.update_target_wrapper_population(pop_new)
            self.pop = self.greedy_selection_population(self.pop, pop_new)
            
        # PHASE2: Defense strategies against predators
        kk = np.random.permutation(self.pop_size)[0]
        pop_new = []
        for idx in range(0, self.pop_size):
            if np.random.rand() < 0.5:
                # S1: the lion attacks the zebra and thus the zebra chooses an escape strategy
                r2 = 0.1
                pos_new = self.pop[idx][self.ID_POS] + r2 * (2 + np.random.rand(self.problem.n_dims) - 1) * (1 - (epoch+1)/self.epoch)*self.pop[idx][self.ID_POS]
            else:
                # S2: other predators attack the zebra and the zebra will choose the offensive strategy
                r2 = np.random.randint(1, 3)
                pos_new = self.pop[idx][self.ID_POS] + np.random.rand(self.problem.n_dims) * (self.pop[kk][self.ID_POS] - r2 * self.pop[idx][self.ID_POS])
            pos_new = self.amend_position(pos_new, self.problem.lb, self.problem.ub)
            pop_new.append([pos_new, None])
            if self.mode not in self.AVAILABLE_MODES:
                target = self.get_target_wrapper(pos_new)
                self.pop[idx] = self.get_better_solution([pos_new, target], self.pop[idx])
        if self.mode in self.AVAILABLE_MODES:
            pop_new = self.update_target_wrapper_population(pop_new)
            self.pop = self.greedy_selection_population(self.pop, pop_new)


In [3]:


# Define a fitness function that evaluates the performance of Random Forest on the selected features
def fitness_function(solution):
    selected_features = [i for i, selected in enumerate(solution) if selected]
    if not selected_features:
        return 0.0  # No features selected, return fitness value of 0
    
    X_subset = X.iloc[:, selected_features]  # Select the corresponding columns from the dataset

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_subset, y, test_size=0.25, random_state=42)

    # Create a Random Forest classifier
    rfc = RandomForestClassifier()

    # Train the classifier
    rfc.fit(X_train, y_train)

    # Predict the labels for the test set
    y_pred = rfc.predict(X_test)

    # Calculate the accuracy score as the fitness value
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

problemOZ = {
    "fit_func": fitness_function,
    "lb": [0] * num_features,  # Lower bound for each feature (0 means not selected)
    "ub": [1] * num_features,  # Upper bound for each feature (1 means selected)
    "minmax": "max", # Maximizing the fitness function
    "save_population": True
}

# Number of optimization runs
num_runs = 10

# Store results
all_results_zoa = []

for run in range(num_runs):
    # Create the PoliticalZOA optimization model
    modelzoa = OriginalZOA(epoch=100, pop_size=100)

    # Measure the total execution time
    start_time = time.time()

    # Solve the optimization problem
    best_position, best_fitness = modelzoa.solve(problemOZ)

    # Calculate and print the total execution time
    end_time = time.time()
    total_time = end_time - start_time
    print(f"Run {run + 1}: Total execution time: {total_time} seconds")

    # Create a dictionary to map feature names to their corresponding position values
    feature_position_dict = {feature_name: position for feature_name, position in zip(X.columns, best_position)}

    # Store results
    all_results_zoa.append({"position_values": feature_position_dict, "fitness": best_fitness})



2023/12/28 08:53:39 AM, INFO, __main__.OriginalZOA: Solving single objective optimization problem.
2023/12/28 08:59:27 AM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 1, Current best: 0.8621926229508197, Global best: 0.8621926229508197, Runtime: 194.22860 seconds
2023/12/28 09:02:41 AM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 2, Current best: 0.8621926229508197, Global best: 0.8621926229508197, Runtime: 193.59550 seconds
2023/12/28 09:06:35 AM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 3, Current best: 0.8621926229508197, Global best: 0.8621926229508197, Runtime: 234.20921 seconds
2023/12/28 09:11:01 AM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 4, Current best: 0.8621926229508197, Global best: 0.8621926229508197, Runtime: 266.65228 seconds
2023/12/28 09:14:37 AM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 5, Current best: 0.8621926229508197, Global best: 0.8621926229508197, Runtime: 215.72031 seconds
2023/12/28 09:17:51 AM, INFO, __main__.OriginalZOA: >Pro

2023/12/28 12:00:53 PM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 49, Current best: 0.8642418032786885, Global best: 0.8642418032786885, Runtime: 193.39472 seconds
2023/12/28 12:04:09 PM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 50, Current best: 0.8642418032786885, Global best: 0.8642418032786885, Runtime: 196.00435 seconds
2023/12/28 12:07:19 PM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 51, Current best: 0.8642418032786885, Global best: 0.8642418032786885, Runtime: 190.53604 seconds
2023/12/28 12:10:29 PM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 52, Current best: 0.8642418032786885, Global best: 0.8642418032786885, Runtime: 189.59788 seconds
2023/12/28 12:13:43 PM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 53, Current best: 0.8642418032786885, Global best: 0.8642418032786885, Runtime: 194.10643 seconds
2023/12/28 12:16:56 PM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 54, Current best: 0.8642418032786885, Global best: 0.8642418032786885, Runtime

2023/12/28 02:40:28 PM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 98, Current best: 0.8642418032786885, Global best: 0.8642418032786885, Runtime: 194.48344 seconds
2023/12/28 02:43:42 PM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 99, Current best: 0.8642418032786885, Global best: 0.8642418032786885, Runtime: 194.00906 seconds
2023/12/28 02:46:58 PM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 100, Current best: 0.8642418032786885, Global best: 0.8642418032786885, Runtime: 196.00331 seconds


Run 1: Total execution time: 21200.671811819077 seconds


2023/12/28 02:46:59 PM, INFO, __main__.OriginalZOA: Solving single objective optimization problem.
2023/12/28 02:51:58 PM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 1, Current best: 0.860655737704918, Global best: 0.860655737704918, Runtime: 192.48922 seconds
2023/12/28 02:55:11 PM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 2, Current best: 0.860655737704918, Global best: 0.860655737704918, Runtime: 193.42542 seconds
2023/12/28 02:58:24 PM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 3, Current best: 0.8611680327868853, Global best: 0.8611680327868853, Runtime: 192.87017 seconds
2023/12/28 03:01:44 PM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 4, Current best: 0.8611680327868853, Global best: 0.8611680327868853, Runtime: 200.10080 seconds
2023/12/28 03:05:08 PM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 5, Current best: 0.8621926229508197, Global best: 0.8621926229508197, Runtime: 204.13793 seconds
2023/12/28 03:08:26 PM, INFO, __main__.OriginalZOA: >Problem

...

Run 9: Total execution time: 14238.266819238663 seconds


2023/12/30 08:56:29 AM, INFO, __main__.OriginalZOA: Solving single objective optimization problem.
2023/12/30 09:00:07 AM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 1, Current best: 0.860655737704918, Global best: 0.860655737704918, Runtime: 142.92470 seconds
2023/12/30 09:02:31 AM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 2, Current best: 0.860655737704918, Global best: 0.860655737704918, Runtime: 144.17359 seconds
2023/12/30 09:04:55 AM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 3, Current best: 0.8621926229508197, Global best: 0.8621926229508197, Runtime: 144.12000 seconds
2023/12/30 09:07:19 AM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 4, Current best: 0.8621926229508197, Global best: 0.8621926229508197, Runtime: 143.54049 seconds
2023/12/30 09:09:48 AM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 5, Current best: 0.8621926229508197, Global best: 0.8621926229508197, Runtime: 149.76818 seconds
2023/12/30 09:12:15 AM, INFO, __main__.OriginalZOA: >Problem

2023/12/30 11:23:02 AM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 49, Current best: 0.8637295081967213, Global best: 0.8637295081967213, Runtime: 216.23260 seconds
2023/12/30 11:26:42 AM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 50, Current best: 0.8637295081967213, Global best: 0.8637295081967213, Runtime: 220.22603 seconds
2023/12/30 11:30:58 AM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 51, Current best: 0.8637295081967213, Global best: 0.8637295081967213, Runtime: 255.87655 seconds
2023/12/30 11:35:13 AM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 52, Current best: 0.8637295081967213, Global best: 0.8637295081967213, Runtime: 254.94281 seconds
2023/12/30 11:39:28 AM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 53, Current best: 0.8637295081967213, Global best: 0.8637295081967213, Runtime: 255.17985 seconds
2023/12/30 11:43:37 AM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 54, Current best: 0.8637295081967213, Global best: 0.8637295081967213, Runtime

2023/12/30 02:20:24 PM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 98, Current best: 0.8637295081967213, Global best: 0.8637295081967213, Runtime: 181.53837 seconds
2023/12/30 02:23:26 PM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 99, Current best: 0.8637295081967213, Global best: 0.8637295081967213, Runtime: 182.34257 seconds
2023/12/30 02:26:26 PM, INFO, __main__.OriginalZOA: >Problem: P, Epoch: 100, Current best: 0.8637295081967213, Global best: 0.8637295081967213, Runtime: 179.20235 seconds


Run 10: Total execution time: 19797.543193101883 seconds


In [7]:
# Print the feature names and corresponding position values for each run in sorted order
for run, result in enumerate(all_results_zoa, start=1):
    sorted_features = sorted(result["position_values"].items(), key=lambda x: x[1])
    print(f"\nRun {run}: Fitness (Accuracy): {result['fitness']}")
    for feature_name, position_value in sorted_features:
        print(f"Feature: {feature_name}, Position Value: {position_value}")


Run 1: Fitness (Accuracy): 0.8642418032786885
Feature: Region, Position Value: 0.11253911418839985
Feature: Family_History, Position Value: 0.1148328348132639
Feature: Gender, Position Value: 0.14800487216344402
Feature: Diet, Position Value: 0.31919352035288256
Feature: Physical_Activity, Position Value: 0.3584642763396105
Feature: Smoking, Position Value: 0.3658945713877244
Feature: BMI, Position Value: 0.46004786111526386
Feature: Waist_Size, Position Value: 0.5866816080472809
Feature: Age, Position Value: 0.5878341084567232
Feature: BP, Position Value: 0.6630453078825618

Run 2: Fitness (Accuracy): 0.8652663934426229
Feature: Smoking, Position Value: 0.1120164143016743
Feature: Gender, Position Value: 0.28120970451413957
Feature: Physical_Activity, Position Value: 0.365301002769429
Feature: Region, Position Value: 0.3713499820017362
Feature: Family_History, Position Value: 0.43991901062563055
Feature: Diet, Position Value: 0.44555526800784095
Feature: BP, Position Value: 0.4763627

In [8]:
# Save the sorted DataFrame to a CSV file
results_df_sorted.to_csv('sorted_results ORIGINAL.csv', index=False)

# Print the feature names and corresponding position values for each run
for rank, (_, result) in enumerate(results_df_sorted.iterrows(), start=1):
    print(f"\nRank {rank}: Fitness (Accuracy): {result['fitness']}")

    # Sort feature names by their position values
    sorted_features = sorted(result['position_values'].items(), key=lambda x: x[1], reverse=True)

    for feature_name, position_value in sorted_features:
        print(f"Feature: {feature_name}, Position Value: {position_value}")


NameError: name 'results_df_sorted' is not defined

In [ ]:
from mealpy.utils.visualize import *

# Draw global best fitness found so far in previous generations
export_convergence_chart(modelOZ.history.list_global_best_fit, title='Global Best Fitness', x_label='epoch', y_label="Accuracy")


# Draw current best fitness in each previous generation
export_convergence_chart(modelOZ.history.list_current_best_fit, title='Local Best Fitness', x_label='epoch', y_label="Accuracy")


# Draw runtime for each generation
export_convergence_chart(modelOZ.history.list_epoch_time, title='Runtime', x_label='epoch', y_label="Second")


# Draw exploration and exploitation chart
export_explore_exploit_chart([modelOZ.history.list_exploration, modelOZ.history.list_exploitation], x_label='epoch')

In [ ]:
## You can access them all via object "history" like this:
modelOZ.history.save_global_objectives_chart(filename="FSF/gocOZ")
modelOZ.history.save_local_objectives_chart(filename="FSF/locOZ")

modelOZ.history.save_global_best_fitness_chart(filename="FSF/gbfcOZ")
modelOZ.history.save_local_best_fitness_chart(filename="FSF/lbfcOZ")

modelOZ.history.save_runtime_chart(filename="FSF/rtcOZ")

modelOZ.history.save_exploration_exploitation_chart(filename="FSF/eecOZ")

modelOZ.history.save_diversity_chart(filename="FSF/dcOZ")



In [ ]:
strong_features = []
moderate_features = []
weak_features = []

for feature_name, position in feature_position_dict.items():
    if position >= 0.6:
        strong_features.append(feature_name)
    elif 0.25 <= position < 0.6:
        moderate_features.append(feature_name)
    else:
        weak_features.append(feature_name)

print("Strong Features:")
for feature in strong_features:
    print(f"Feature: {feature}, Position Value: {feature_position_dict[feature]}")

print("\nModerate Features:")
for feature in moderate_features:
    print(f"Feature: {feature}, Position Value: {feature_position_dict[feature]}")

print("\nWeak Features:")
for feature in weak_features:
    print(f"Feature: {feature}, Position Value: {feature_position_dict[feature]}")


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Assuming you have your dataset X and target variable y defined

# Create datasets with moderate and strong features
X_moderate = X[list(moderate_features)]
X_strong = X[list(strong_features)]

# Split the data into training and testing sets for moderate features
X_moderate_train, X_moderate_test, y_train, y_test = train_test_split(X_moderate, y, test_size=0.25, random_state=42)

# Create a Random Forest classifier for moderate features
rfc_moderate = RandomForestClassifier()
rfc_moderate.fit(X_moderate_train, y_train)
y_pred_moderate = rfc_moderate.predict(X_moderate_test)
accuracy_moderate = accuracy_score(y_test, y_pred_moderate)

# Split the data into training and testing sets for strong features
X_strong_train, X_strong_test, y_train, y_test = train_test_split(X_strong, y, test_size=0.25, random_state=42)

# Create a Random Forest classifier for strong features
rfc_strong = RandomForestClassifier()
rfc_strong.fit(X_strong_train, y_train)
y_pred_strong = rfc_strong.predict(X_strong_test)
accuracy_strong = accuracy_score(y_test, y_pred_strong)

print(f"Accuracy for Moderate Features: {accuracy_moderate}")
print(f"Accuracy for Strong Features: {accuracy_strong}")


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Assuming you have your dataset X and target variable y defined

# Create datasets with moderate and strong features
X_moderate = X[list(moderate_features)]
X_strong = X[list(strong_features)]

# Combine the moderate and strong features into one feature set
X_combined = pd.concat([X_moderate, X_strong], axis=1)

# Split the data into training and testing sets for the combined features
X_combined_train, X_combined_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.25, random_state=42)

# Create a Random Forest classifier for the combined features
rfc_combined = RandomForestClassifier()
rfc_combined.fit(X_combined_train, y_train)
y_pred_combined = rfc_combined.predict(X_combined_test)
accuracy_combined = accuracy_score(y_test, y_pred_combined)

print(f"Accuracy for Combined Moderate and Strong Features: {accuracy_combined}")


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Assuming you have your dataset X and target variable y defined

# Split the data into training and testing sets for the three cases
X_moderate_train, X_moderate_test, y_train, y_test = train_test_split(X[list(moderate_features)], y, test_size=0.25, random_state=42)
X_strong_train, X_strong_test, _, _ = train_test_split(X[list(strong_features)], y, test_size=0.25, random_state=42)
X_combined_train, X_combined_test, _, _ = train_test_split(X_combined, y, test_size=0.25, random_state=42)

# Create Random Forest classifiers for the three cases
rfc_moderate = RandomForestClassifier()
rfc_strong = RandomForestClassifier()
rfc_combined = RandomForestClassifier()

# Fit the models
rfc_moderate.fit(X_moderate_train, y_train)
rfc_strong.fit(X_strong_train, y_train)
rfc_combined.fit(X_combined_train, y_train)

# Predict the labels for the test sets
y_pred_moderate = rfc_moderate.predict(X_moderate_test)
y_pred_strong = rfc_strong.predict(X_strong_test)
y_pred_combined = rfc_combined.predict(X_combined_test)

# Calculate metrics for each case
precision_moderate = precision_score(y_test, y_pred_moderate)
recall_moderate = recall_score(y_test, y_pred_moderate)
f1_moderate = f1_score(y_test, y_pred_moderate)
accuracy_moderate = accuracy_score(y_test, y_pred_moderate)

precision_strong = precision_score(y_test, y_pred_strong)
recall_strong = recall_score(y_test, y_pred_strong)
f1_strong = f1_score(y_test, y_pred_strong)
accuracy_strong = accuracy_score(y_test, y_pred_strong)

precision_combined = precision_score(y_test, y_pred_combined)
recall_combined = recall_score(y_test, y_pred_combined)
f1_combined = f1_score(y_test, y_pred_combined)
accuracy_combined = accuracy_score(y_test, y_pred_combined)

# Print metrics for each case
print("Metrics for Moderate Features:")
print(f"Precision: {precision_moderate}")
print(f"Recall: {recall_moderate}")
print(f"F1 Score: {f1_moderate}")
print(f"Accuracy: {accuracy_moderate}")

print("\nMetrics for Strong Features:")
print(f"Precision: {precision_strong}")
print(f"Recall: {recall_strong}")
print(f"F1 Score: {f1_strong}")
print(f"Accuracy: {accuracy_strong}")

print("\nMetrics for Combined Moderate and Strong Features:")
print(f"Precision: {precision_combined}")
print(f"Recall: {recall_combined}")
print(f"F1 Score: {f1_combined}")
print(f"Accuracy: {accuracy_combined}")





In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming you have calculated precision, recall, F1 score, and accuracy for the three cases

# Metrics and labels for the three cases
metrics = ['Precision', 'Recall', 'F1 Score', 'Accuracy']
cases = ['Moderate Features', 'Strong Features', 'Moderate & Strong Features']

# Values for each metric
precision_values = [precision_moderate, precision_strong, precision_combined]
recall_values = [recall_moderate, recall_strong, recall_combined]
f1_values = [f1_moderate, f1_strong, f1_combined]
accuracy_values = [accuracy_moderate, accuracy_strong, accuracy_combined]

# Set the width of the bars
bar_width = 0.2

# Set the position of the bars on the x-axis
r1 = np.arange(len(cases))
r2 = [x + bar_width for x in r1]
r3 = [x + bar_width for x in r2]
r4 = [x + bar_width for x in r3]

# Create the grouped bar chart
plt.bar(r1, precision_values, width=bar_width, label='Precision', alpha=0.7)
plt.bar(r2, recall_values, width=bar_width, label='Recall', alpha=0.7)
plt.bar(r3, f1_values, width=bar_width, label='F1 Score', alpha=0.7)
plt.bar(r4, accuracy_values, width=bar_width, label='Accuracy', alpha=0.7)

# Set the x-axis labels
plt.xlabel('Feature Cases')
plt.xticks([r + bar_width for r in range(len(cases))], cases)

# Set the y-axis label
plt.ylabel('Scores')

# Add a legend
plt.legend(loc='lower left')

# Set the title
plt.title('Metrics for Different Feature Cases')

# Annotate the bars with numeric values
for i in range(len(cases)):
    for j, metric_value in enumerate([precision_values, recall_values, f1_values, accuracy_values]):
        plt.text(i + (j * bar_width) - 0.07, metric_value[i] + 0.02, f"{metric_value[i]:.2f}", fontsize=8, fontweight='bold')
# Save the figure as an image file (e.g., PNG)
plt.savefig('metrics_bar_chart_OZ.png')
# Show the plot
plt.show()


In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# Assuming you have already trained the classifiers and obtained probability predictions
# y_score should contain the predicted probabilities for the positive class

def plot_roc(y_true, y_score, case_label):
    fpr, tpr, _ = roc_curve(y_true, y_score)
    roc_auc = auc(fpr, tpr)

    plt.figure()
    lw = 2
    plt.plot(fpr, tpr, color='darkorange', lw=lw, label=f'ROC curve (area = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curve for {case_label}')
    plt.legend(loc='lower right')
                # Save the figure as an image file (e.g., PNG)
    plt.savefig(f'roc_curve_FSOZ_{case_label}.png')
    plt.show()

# Assuming you have y_true and y_score for each classifier
plot_roc(y_test, y_pred_moderate, 'Moderate Features')
plot_roc(y_test, y_pred_strong, 'Strong Features')
plot_roc(y_test, y_pred_combined, 'Combined Moderate and Strong Features')


In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Function to plot confusion matrix
def plot_confusion_matrix(y_true, y_pred, case_label):
    cm = confusion_matrix(y_true, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(cmap='Blues', values_format='d')
    plt.title(f'Confusion Matrix for {case_label}')
                    # Save the figure as an image file (e.g., PNG)
    plt.savefig(f'confusion_matrix_FSOZ_{case_label}.png')
    plt.show()

# Assuming you have y_pred for each classifier
plot_confusion_matrix(y_test, y_pred_moderate, 'Moderate Features')
plot_confusion_matrix(y_test, y_pred_strong, 'Strong Features')
plot_confusion_matrix(y_test, y_pred_combined, 'Combined Moderate and Strong Features')


In [ ]:
import time
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, auc
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Convert binary feature selection to column names
selected_columns = [X.columns[i] for i, selected in enumerate(best_position) if selected]

# Evaluate the best solution found by the metaheuristic algorithm
X_best = X[selected_columns]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_best, y, test_size=0.25, random_state=42)

# Create a Random Forest classifier
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)

# Predict the labels for the test set
y_pred = rfc.predict(X_test)

# Calculate and print the accuracy score
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy (Fitness): {accuracy}")

# Compute ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.savefig('roc_curve OZ.png')  # Save the ROC curve figure
# Compute and plot the confusion matrix
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(8, 6))
plt.imshow(cm, interpolation='nearest', cmap=plt.get_cmap('Blues'))
plt.title('Confusion Matrix')
plt.colorbar()
classes = ['Negative', 'Positive']
tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes)
plt.yticks(tick_marks, classes)
thresh = cm.max() / 2.
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, format(cm[i, j], 'd'), horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.savefig('confusion_matrixOZ.png')  # Save the confusion matrix figure

plt.show()


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Calculate Accuracy
accuracy = accuracy_score(y_test, y_pred)

# Calculate Precision
precision = precision_score(y_test, y_pred)

# Calculate Recall
recall = recall_score(y_test, y_pred)

# Calculate F1 Score
f1 = f1_score(y_test, y_pred)

# You already have the ROC curve and AUC
auc = roc_auc

# Store the metrics in a dictionary
metrics_dict = {
    "Accuracy": accuracy,
    "Precision": precision,
    "Recall": recall,
    "F1 Score": f1,
    "AUC": auc
}

# Print the dictionary
for metric, value in metrics_dict.items():
    print(f"{metric}: {value:.2f}")

    

    


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt

# Calculate the metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# You already have the AUC value
auc = roc_auc

# Create a dictionary to store the metrics
metrics_dict = {
    "Accuracy": accuracy,
    "Precision": precision,
    "Recall": recall,
    "F1 Score": f1,
    "AUC": auc
}

# Plot the metrics as a bar chart with values on top of the bars
plt.figure(figsize=(8, 5))  # Adjust the figsize here (e.g., 8x5)
bars = plt.bar(metrics_dict.keys(), metrics_dict.values(), color='darkorange')
plt.xlabel('Metrics')
plt.ylabel('Value')
plt.title('Model Evaluation Metrics')
plt.xticks(rotation=45)

# Add values on top of the bars
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, yval, f'{yval:.2f}', ha='center', va='bottom')

plt.savefig('Performance_metrics_plot_without_FS.png', bbox_inches='tight')  # Adjust the filename
plt.show()


In [ ]:
# # Save the results to a CSV file
# results_df = pd.DataFrame.from_dict(feature_position_dict, orient='index', columns=['Position Value'])
# results_df.index.name = 'Feature Name'
# results_df.to_csv('feature_selection_resultsOZ.csv')

In [ ]:
import numpy as np
import time  # Import the time module

from mealpy.utils import io
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Assuming you have a dataset 'X' and corresponding labels 'y'

# Determine the number of features
num_features = X.shape[1]

# Define a fitness function that evaluates the performance of Random Forest on the selected features
def fitness_function(solution):
    selected_features = [i for i, selected in enumerate(solution) if selected]
    
    if not selected_features:
        return 0.0  # No features selected, return fitness value of 0
    
    X_subset = X[:, selected_features]  # Select the corresponding columns from the dataset

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_subset, y, test_size=0.25, random_state=42)

    # Create a Random Forest classifier
    rfc = RandomForestClassifier()

    # Train the classifier
    rfc.fit(X_train, y_train)

    # Predict the labels for the test set
    y_pred = rfc.predict(X_test)

    # Calculate the accuracy score as the fitness value
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

problem = {
    "fit_func": fitness_function,
    "lb": [0] * num_features,  # Lower bound for each feature (0 means not selected)
    "ub": [1] * num_features,  # Upper bound for each feature (1 means selected)
    "minmax": "max",           # Maximizing the fitness function
}

# epoch = 100
# pop_size = 100
# model = OriginalZOA(epoch, pop_size)

# # Measure the total execution time
# start_time = time.time()

# best_position, best_fitness = model.solve(problem)
# print(f"Solution: {best_position}, Fitness: {best_fitness}")

# # Calculate and print the total execution time
# end_time = time.time()
# total_time = end_time - start_time
# print(f"Total execution time: {total_time} seconds")


In [ ]:

from mealpy.tuner import Tuner
### Define model and parameter grid of the model (just like ParameterGrid / GridSearchCV)
model = OriginalZOA()

paras_OriginalZOA_grid = {
    "epoch": [50,75,100],
    "pop_size": [50,75,100]
    
}

### Define the Tuner and run it
if __name__ == "__main__":
    tuner = Tuner(model, paras_OriginalZOA_grid)

        ## Try to run this optimizer on this problem 10 times (n_trials = 10).
    ## Get the best model by mean value of all trials
    tuner.execute(problem=problem, n_trials=5, mode="parallel", n_workers=4)

    ## Better to save the tunning results to CSV for later usage
    tuner.export_results("history/Orginal", save_as="csv")

    ## Print out the best pameters
    print(f"Best parameter: {tuner.best_params}")

    ## Print out the best score of the best parameter
    print(f"Best score: {tuner.best_score}")

    ## Print out the algorithm with the best parameter
    print(f"Best Optimizer: {tuner.best_algorithm}")


    ## Now we can even re-train the algorithm with the best parameter by calling resolve() function
    ## Resolve() function will call the solve() function in algorithm with default problem parameter is removed.
    ##    other parameters of solve() function is keeped and can be used.

    best_position, best_fitness = tuner.resolve()
    print(f"Best solution after re-solve: {best_position}")
    print(f"Best fitness after re-solve: {best_fitness}")

In [ ]:
# Define the number of rounds
num_rounds = 5
results = []

# Run optimization for multiple rounds
for round in range(num_rounds):
    # Measure the total execution time for each round
    start_time = time.time()

    # Run the optimization
    best_position, best_fitness = model.solve(problem)
    results.append((best_position, best_fitness))

    # Calculate and print the total execution time for each round
    end_time = time.time()
    total_time = end_time - start_time
    print(f"Round {round + 1}: Solution: {best_position}, Fitness: {best_fitness}")
    print(f"Total execution time: {total_time} seconds")

# Print the results of all rounds
for round, (best_position, best_fitness) in enumerate(results):
    print(f"Round {round + 1}: Solution: {best_position}, Fitness: {best_fitness}")